<a href="https://colab.research.google.com/github/sakusakupanda03/BachelorThesis/blob/main/notebooks/01_collect_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import glob

path = "/content/drive/MyDrive/sotsuron/data"

In [ ]:
variable_dict = {
                 "Market Price (USD)": ["Ether Daily Price (USD) Chart", "market-price.csv", "export-EtherPrice.csv"],
                 "Market Capitalization (USD)": ["Ether Market Capitalization Chart", "market-cap.csv", "export-MarketCap.csv"],
                 "Average Block Size (MB)": ["Ethereum Average Block Size Chart", "avg-block-size.csv", "export-BlockSize.csv"],
                 "Average Confirmation Time": ["Average Block Time Chart", "avg-confirmation-time.csv", "export-BlockTime.csv"],
                 "Total Hash Rate (TH/s)": ["Ethereum Network Hash Rate Chart", "hash-rate.csv", "export-NetworkHash.csv"],
                 "Network Difficulty": ["Ethereum Network Difficulty Chart", "difficulty.csv", "export-BlockDifficulty.csv"],
                 "Miners Revenue (USD)": ["Ethereum Daily Gas Used Chart", "miners-revenue.csv", "export-GasUsed.csv"],
                 "Total Transaction Fees (BTC)": ["Ethereum Network Transaction Fee Chart", "transaction-fees.csv", "export-TransactionFee.csv"],
                 "Fees Per Transaction (USD)": ["Average Transaction Fee Chart", "fees-usd-per-transaction.csv", "export-AverageDailyTransactionFee.csv"],
                 "Unique Addresses Used": ["Active Ethereum Addresses", "n-unique-addresses.csv", "export-DailyActiveEthAddress.csv"],
                 "Confirmed Transactions Per Day": ["Ethereum Daily Verified Contracts Chart", "n-transactions.csv", "export-verified-contracts.csv"]}
btc_variable_list = list(variable_dict.keys())
eth_variable_list = [list(variable_dict.values())[i][0] for i in range(len(variable_dict))]
btc_filename_list = [list(variable_dict.values())[i][1] for i in range(len(variable_dict))]
eth_filename_list = [list(variable_dict.values())[i][2] for i in range(len(variable_dict))]

btc_csv_list = glob.glob(path+"/01_raw/btc/*.csv")
eth_csv_list = glob.glob(path+"/01_raw/eth/*.csv")

# Data collection

In [ ]:
# BTC

# Market cap needs speacial operation due to its difference of time interval causing the inconsistency with the other variables
btc_market_cap = pd.read_csv("/content/drive/MyDrive/sotsuron/data/01_raw/btc/" +"market-cap.csv")
btc_market_cap["Timestamp"] = pd.to_datetime(btc_market_cap["Timestamp"]).round("D") + datetime.timedelta(days=1)
btc_market_cap = btc_market_cap.groupby("Timestamp").mean()

# Merge all the variables
btc_merged = btc_market_cap.copy()
for b in btc_filename_list:
    if b == "market-cap.csv":
        continue
    else: 
        btc_variable = pd.read_csv("/content/drive/MyDrive/sotsuron/data/01_raw/btc/" + b)
        btc_variable["Timestamp"] = pd.to_datetime(btc_variable["Timestamp"])
        btc_variable = btc_variable.set_index("Timestamp", drop=True)
        btc_merged = pd.merge(btc_merged, btc_variable, left_index=True, right_index=True, how="outer")

btc_merged = btc_merged[: "2022-08-11"]
btc_merged.to_pickle(path + "/02_intermediate/btc.pickle")

In [ ]:
btc_merged.index

DatetimeIndex(['2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
               '2019-08-17', '2019-08-18', '2019-08-19', '2019-08-20',
               '2019-08-21', '2019-08-22',
               ...
               '2022-08-02', '2022-08-03', '2022-08-04', '2022-08-05',
               '2022-08-06', '2022-08-07', '2022-08-08', '2022-08-09',
               '2022-08-10', '2022-08-11'],
              dtype='datetime64[ns]', name='Timestamp', length=1095, freq=None)

In [ ]:
eth_filename_list

['export-EtherPrice.csv',
 'export-MarketCap.csv',
 'export-BlockSize.csv',
 'export-BlockTime.csv',
 'export-NetworkHash.csv',
 'export-BlockDifficulty.csv',
 'export-GasUsed.csv',
 'export-TransactionFee.csv',
 'export-AverageDailyTransactionFee.csv',
 'export-DailyActiveEthAddress.csv',
 'export-verified-contracts.csv']

In [ ]:
# ETH
eth_merged = pd.read_csv("/content/drive/MyDrive/sotsuron/data/01_raw/eth/" + eth_filename_list[0])
eth_merged["Date(UTC)"] = pd.to_datetime(eth_merged["Date(UTC)"])
eth_merged = eth_merged.set_index("Date(UTC)", drop=True)
eth_merged = eth_merged.rename({"Value": eth_filename_list[0].strip(".csv")}, axis=1)
eth_merged = eth_merged.drop("UnixTimeStamp", axis=1)

for e in eth_filename_list[1:]:
    eth_variable = pd.read_csv("/content/drive/MyDrive/sotsuron/data/01_raw/eth/" + e)
    
    eth_variable["Date(UTC)"] = pd.to_datetime(eth_variable["Date(UTC)"])
    eth_variable = eth_variable.set_index("Date(UTC)", drop=True)

    if "Value" in eth_variable.columns:
        eth_variable = eth_variable.rename({"Value": e.strip(".csv")}, axis=1)
    if "UnixTimeStamp" in eth_variable.columns:
        eth_variable = eth_variable.drop("UnixTimeStamp", axis=1)
    
    
    eth_merged = pd.merge(eth_merged, eth_variable, left_index=True, right_index=True, how="outer")

eth_merged = eth_merged.rename({"Average Txn Fee (USD)": "export-AverageDailyTransactionFee", "Unique Address Total Count": "export-DailyActiveEthAddress", "MarketCap": "export-MarketCap", "No. of Verified Contracts": "export-verified-contracts"}, axis=1)
eth_merged = eth_merged.drop(["Supply", "Price", "Average Txn Fee (Ether)", "Unique Address Receive Count", "Unique Address Sent Count", "DateTime"], axis=1)
eth_merged = eth_merged[btc_merged.index[0]:btc_merged.index[-1]]
eth_merged.to_pickle(path + "/02_intermediate/eth.pickle")